In [ ]:
# STEP 1: Install Kaggle CLI
!pip install -q kaggle

# STEP 2: Upload kaggle.json
from google.colab import files
uploaded = files.upload()  # Upload kaggle.json when prompted

# STEP 3: Fix filename (in case it was auto-renamed)
import shutil
if "kaggle (1).json" in uploaded:
    shutil.move("kaggle (1).json", "kaggle.json")

# STEP 4: Set up Kaggle API key
import os
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.copy("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)

# STEP 5: Download the dataset
!kaggle competitions download -c aptos2019-blindness-detection

# STEP 6: Unzip the downloaded files
!unzip -q aptos2019-blindness-detection.zip
!unzip -q train_images.zip

# STEP 7: Preprocessing
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load CSV
df = pd.read_csv("train.csv")
df["diagnosis"] = df["diagnosis"].astype(str)  # Needed for categorical

df = pd.read_csv("train.csv")

# Add ".png" extension to all id_codes
df["id_code"] = df["id_code"].apply(lambda x: x + ".png")


# Sample small dataset for quick testing (you can remove this line later)
df = df.sample(300, random_state=42)

# Split data
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['diagnosis'])

# Image generator
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20
)

train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='train_images/',
    x_col='id_code',
    y_col='diagnosis',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16
)

val_gen = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='train_images/',
    x_col='id_code',
    y_col='diagnosis',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16,
    shuffle=False
)

# STEP 8: Model Building
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# STEP 9: Train Model
early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[early])

# STEP 10: Plot Accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

# STEP 11: Save model
model.save("dr_model.h5")

Saving kaggle.json to kaggle (1).json
^C
unzip:  cannot find or open aptos2019-blindness-detection.zip, aptos2019-blindness-detection.zip.zip or aptos2019-blindness-detection.zip.ZIP.
unzip:  cannot find or open train_images.zip, train_images.zip.zip or train_images.zip.ZIP.


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'